In [1]:
import os
import numpy as np
from PIL import Image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [18]:
# Set the path to the MSTAR-11 dataset directory
dataset_dir = ''

# Set the input image size for GoogleNet
input_size = (128, 128)

# Set the number of classes in your dataset
num_classes = 11

# Load the images and labels from the dataset
images = []
labels = []
for folder in os.listdir(os.path.join(dataset_dir, 'train')):
    folder_path = os.path.join(dataset_dir, 'train', folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).resize(input_size)
            images.append(np.array(image))
            labels.append(folder)


In [19]:
# Convert the images and labels to numpy arrays

images = np.array(images)
labels = np.array(labels)

In [20]:



# Perform one-hot encoding on the labels
label_encoder = LabelEncoder()
integer_labels = label_encoder.fit_transform(labels)
onehot_encoder = OneHotEncoder(sparse=False)
integer_labels = integer_labels.reshape(len(integer_labels), 1)
onehot_labels = onehot_encoder.fit_transform(integer_labels)

# Preprocess the images
preprocessed_images = preprocess_input(images)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_images, onehot_labels, test_size=0.2, random_state=42)



c:\Users\kaushal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
# Load the pretrained GoogleNet model
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
print(X_train.shape)  # Verify the shape of X_train
print(y_train.shape)  # Verify the shape of y_train


(2436, 128, 128)
(2436, 11)


In [26]:
X_train_rgb = np.repeat(X_train[..., np.newaxis], 3, -1)

In [27]:
# Train the model
model.fit(X_train_rgb, y_train, batch_size=32, epochs=10)

Epoch 1/10
77/77 [==============================] - 198s 2s/step - loss: 1.3574 - accuracy: 0.5686
Epoch 2/10
77/77 [==============================] - 189s 2s/step - loss: 0.6915 - accuracy: 0.7943
Epoch 3/10
77/77 [==============================] - 181s 2s/step - loss: 0.3970 - accuracy: 0.8822
Epoch 4/10
77/77 [==============================] - 182s 2s/step - loss: 0.4937 - accuracy: 0.8563
Epoch 5/10
77/77 [==============================] - 183s 2s/step - loss: 0.3102 - accuracy: 0.9076
Epoch 6/10
77/77 [==============================] - 186s 2s/step - loss: 0.2128 - accuracy: 0.9380
Epoch 7/10
77/77 [==============================] - 184s 2s/step - loss: 0.1866 - accuracy: 0.9511
Epoch 8/10
77/77 [==============================] - 182s 2s/step - loss: 0.2176 - accuracy: 0.9446
Epoch 9/10
77/77 [==============================] - 183s 2s/step - loss: 0.3164 - accuracy: 0.9105
Epoch 10/10
77/77 [==============================] - 183s 2s/step - loss: 0.1078 - accuracy: 0.9692


In [40]:
# Evaluate the model on the testing set
X_test_rgb = np.repeat(X_test[..., np.newaxis], 3, -1)
y_pred = model.predict(X_test_rgb)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test


20/20 [==============================] - 10s 493ms/step


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [41]:
# Convert one-hot encoded labels back to integer labels
y_test_classes = np.argmax(y_test, axis=1)

In [38]:
print(X_test.shape)           # Shape of the testing dataset
print(y_test_classes.shape)   # Shape of the true labels for testing
print(y_pred_classes.shape)

(609, 128, 128)
(609,)
(2436,)


In [42]:



# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred_classes)
print('Accuracy:', accuracy)


Accuracy: 0.7339901477832512
